In [2]:
from groq import Groq
from llama_index.core import Settings, ServiceContext, StorageContext, SimpleDirectoryReader, PropertyGraphIndex
from llama_index.llms.groq import Groq as Groq_llamaindex
from llama_index.llms.openai import OpenAI
from llama_index.llms.replicate import Replicate
from llama_index.embeddings.huggingface import HuggingFaceEmbedding
from llama_index.graph_stores.neo4j import Neo4jGraphStore, Neo4jPropertyGraphStore
from llama_index.core.indices.property_graph import VectorContextRetriever
from llama_index.core import VectorStoreIndex
from llama_index.core.agent import ReActAgent, FunctionCallingAgentWorker, AgentRunner
from llama_index.core.tools import BaseTool, FunctionTool
from milvus import default_server
from dotenv import load_dotenv
import json
import os
import numpy as np
import os
BASE_ENTITY_LABEL = "__Entity__"
BASE_NODE_LABEL = "__Node__"

# Retrieve API keys and credentials securely
GROQ_API_KEY = os.getenv('GROQ_API_KEY')
NEO4J_USERNAME = os.getenv('NEO4J_USERNAME')
NEO4J_PASSWORD = os.getenv('NEO4J_PASSWORD')
NEO4J_URL = os.getenv('NEO4J_URL', 'bolt://localhost:7687')
NEO4J_DATABASE = os.getenv('NEO4J_DATABASE', 'neo4j')
REPLICATE_API_KEY = os.getenv('REPLICATE_API_KEY')
os.environ["REPLICATE_API_KEY"] = REPLICATE_API_KEY

#Initialize the Replicate class
llm = Replicate(
    model="meta/meta-llama-3-70b-instruct"
)

Settings.llm = llm
Settings.embed_model = HuggingFaceEmbedding(
    model_name="BAAI/bge-small-en-v1.5"
)

ServiceContext.llm = llm

import logging
import sys

logging.basicConfig(stream=sys.stdout, level=logging.INFO)
logging.getLogger().addHandler(logging.StreamHandler(stream=sys.stdout))

property_graph_store = Neo4jPropertyGraphStore(
    username=NEO4J_USERNAME,
    password=NEO4J_PASSWORD,
    url=NEO4J_URL,
    database=NEO4J_DATABASE,
)
storage_context = StorageContext.from_defaults(property_graph_store=property_graph_store)


index = PropertyGraphIndex.from_existing(
    property_graph_store=property_graph_store,
    llm=llm,
    embed_model=Settings.embed_model,
)


Received notification from DBMS server: {severity: WARNING} {code: Neo.ClientNotification.Statement.FeatureDeprecationWarning} {category: DEPRECATION} {title: This feature is deprecated and will be removed in future versions.} {description: The procedure has a deprecated field. ('config' used by 'apoc.meta.graphSample' is deprecated.)} {position: line: 1, column: 1, offset: 0} for query: "CALL apoc.meta.graphSample() YIELD nodes, relationships RETURN nodes, [rel in relationships | {name:apoc.any.property(rel, 'type'), count: apoc.any.property(rel, 'count')}] AS relationships"
INFO:neo4j.notifications:Received notification from DBMS server: {severity: INFORMATION} {code: Neo.ClientNotification.Schema.IndexOrConstraintAlreadyExists} {category: SCHEMA} {title: `CREATE CONSTRAINT IF NOT EXISTS FOR (e:__Node__) REQUIRE (e.id) IS UNIQUE` has no effect.} {description: `CONSTRAINT constraint_ec67c859 FOR (e:__Node__) REQUIRE (e.id) IS UNIQUE` already exists.} {position: None} for query: 'CREAT

In [3]:
class DummnQuery:
    def __init__(self, query_str, embedding):
        self.query_str = query_str
        self.filters = None
        self.query_embedding = embedding
        self.similarity_top_k = 4



NameError: name 'query_str' is not defined

In [4]:
# Auto evaluations
import pandas as pd

# reaad question from file questions.csv

questions = pd.read_csv("oia_files_and_questions.csv")




In [20]:
query.__dict__

{'query_str': 'What is the cost of the Road to Zero advertising campaign as of 30 June 2022?',
 'filters': None,
 'query_embedding': [-0.021242610094222156,
  -0.004481015478974426,
  -0.015735820192819487,
  0.0025441485576939396,
  -0.021744331441722907,
  -0.008412694417045289,
  0.01996731777016267,
  0.04988253726200624,
  0.020295695571059532,
  -0.01403018135982626,
  0.007409222798978353,
  -0.005565413522576405,
  0.015617027223358148,
  0.006199745383104551,
  0.054479141393071646,
  0.02045437014909679,
  0.012769927675896263,
  0.0351054903555226,
  -0.06252808706810722,
  0.04160235333558801,
  0.04858569094499984,
  -0.014575563498180021,
  0.036938678866324876,
  -0.013594121282146154,
  -0.010002352027812151,
  0.029601157122811715,
  0.040068501521917904,
  -0.02222427168812651,
  -0.024153495918015962,
  -0.13550690329307086,
  -0.027209421495798154,
  0.002005890229499185,
  0.037600692464401585,
  0.03974794511767951,
  -0.004128028120083565,
  0.0010615816149352627

In [23]:
question

'How effective was the Road to Zero advertising campaign in terms of reach and public support as of June 2022?'

In [39]:
query_str

File Name                        oia-12662-response-letter.pdf
Question     Are there any future upgrades planned to addre...
Answer       The speed limit in Featherston was reduced to ...
Name: 3, dtype: object

In [51]:
questions[['Question']].iloc[3].values[0]

'Are there any future upgrades planned to address crash risks at the SH2/SH53 intersection?'

In [53]:
query_str = questions[['Question']].iloc[3].values[0]
print(query_str)
embedding = index_retriever.sub_retrievers[0]._embed_model.get_agg_embedding_from_queries([query_str])
query = DummnQuery(query_str, embedding)
print(query.query_str)
nodes_with_score = property_graph_store.vector_query(query)
ids = []
for node in nodes_with_score[0]:
    ids.append(node.name)
ids



Are there any future upgrades planned to address crash risks at the SH2/SH53 intersection?


Batches: 100%|██████████| 1/1 [00:00<00:00, 17.76it/s]

Are there any future upgrades planned to address crash risks at the SH2/SH53 intersection?


['Request for information on Hornby Intersection improvements',
 'no other intersection safety improvements planned',
 'intersection of SH2 and SH53',
 'improving the safety performance of the vehicle fleet']

In [37]:
i

3

In [33]:
query.__dict__

{'query_str': 'Are there any future upgrades planned to address crash risks at the SH2/SH53 intersection?',
 'filters': None,
 'query_embedding': [-0.022514624464222126,
  -0.005588627276786913,
  -0.013872995926067232,
  0.003933403496436464,
  -0.021763937201143967,
  -0.008514262972312281,
  0.019144291562649112,
  0.050371094689601,
  0.023080528121338123,
  -0.010546273638546053,
  0.006164791877381504,
  -0.0032071551961255157,
  0.017762556570556223,
  0.0036423304163488665,
  0.054896473636229834,
  0.018663538048147327,
  0.010819969336605735,
  0.038634341564546856,
  -0.06262155065519942,
  0.042988656937248175,
  0.04795753604008092,
  -0.013865082385018468,
  0.03727544931591385,
  -0.015426774165179166,
  -0.009988360672044413,
  0.028946460395430525,
  0.039812263308299914,
  -0.0246941054188129,
  -0.026966987125989463,
  -0.13417760729789735,
  -0.02519017968637248,
  0.003225379225073589,
  0.040451942591203584,
  0.04225497498280472,
  -0.00441938364205675,
  0.00019

In [45]:
query_str = question
print(query_str)
embedding = index_retriever.sub_retrievers[0]._embed_model.get_agg_embedding_from_queries(query_str)
query = DummnQuery(query_str, embedding)

print(query.query_str)
nodes_with_score = property_graph_store.vector_query(query)
ids = []
for node in nodes_with_score[0]:
    ids.append(node.name)

print("this are the ids")
for id in ids:
    print(id)

Are there any future upgrades planned to address crash risks at the SH2/SH53 intersection?


Batches: 100%|██████████| 1/1 [00:00<00:00, 106.85it/s]


Are there any future upgrades planned to address crash risks at the SH2/SH53 intersection?
this are the ids
discussions
verbatim staff comments from 'AskYourTeam' Surveys
comment
Email chain discussion


In [55]:
for current_depth in range(1, 7):
    similarity_top_k = 4    
    path_depth = current_depth
    vector_retriever = VectorContextRetriever(
    index.property_graph_store,
    # only needed when the graph store doesn't support vector queries
    # vector_store=index.vector_store,
    embed_model=Settings.embed_model,
    # include source chunk text with retrieved paths
    include_text=True,
    # the number of nodes to fetch
    similarity_top_k=similarity_top_k,
    # the depth of relations to follow after node retrieval
    path_depth=path_depth,
    
  
    )

    #retriever = index.as_retriever(sub_retrievers=[vector_retriever])
    index_query_engine = index.as_query_engine(sub_retrievers=[vector_retriever])
    index_retriever = index.as_retriever(sub_retrievers=[vector_retriever])

    import nest_asyncio
    nest_asyncio.apply()
    
    for  i, question in enumerate(questions['Question']):
        

       
        query_str = question
        print(query_str)
        embedding = index_retriever.sub_retrievers[0]._embed_model.get_agg_embedding_from_queries([query_str])
        query = DummnQuery(query_str, embedding)

    
        print(query.query_str)
        nodes_with_score = property_graph_store.vector_query(query)
        ids = []
        for node in nodes_with_score[0]:
            ids.append(node.name)
     

        response_unsorted = property_graph_store.structured_query(
        f"""
        WITH $ids AS id_list
        UNWIND range(0, size(id_list) - 1) AS idx
        MATCH (e:`{BASE_ENTITY_LABEL}`)
        WHERE e.id = id_list[idx]
        MATCH p=(e)-[r*1..{path_depth}]-(other)
        WHERE ALL(rel IN relationships(p) WHERE type(rel) <> 'MENTIONS')
        UNWIND relationships(p) AS rel
        WITH DISTINCT rel, idx, id_list[idx] AS seed_id, vector.similarity.cosine(rel.embedding, $embedding) AS rel_score
        WITH startNode(rel) AS source,
            type(rel) AS type,
            rel{{.*}} AS rel_properties,
            endNode(rel) AS endNode,
            idx,
            rel_score,
            seed_id
        LIMIT toInteger($limit)
        RETURN source.id AS source_id, 
            [l IN labels(source) WHERE NOT l IN ['{BASE_ENTITY_LABEL}', '{BASE_NODE_LABEL}'] | l][0] AS source_type,
            source{{.*, embedding: Null, id: Null}} AS source_properties,
            type,
            rel_properties,
            endNode.id AS target_id, 
            [l IN labels(endNode) WHERE NOT l IN ['{BASE_ENTITY_LABEL}', '{BASE_NODE_LABEL}'] | l][0] AS target_type,
            endNode{{.*, embedding: Null, id: Null}} AS target_properties,
            idx,
            rel_score,
            seed_id
        ORDER BY idx
        LIMIT toInteger($limit)
        """,
        param_map={"ids": ids, "limit": 30, "embedding": query.query_embedding},
    )
        
        response_sorted = property_graph_store.structured_query(f"""
        // Assign an index to each id in the original list
        WITH [i IN RANGE(0, size($ids)-1) | {{id: $ids[i], seed_idx: i}}] AS id_list
        UNWIND id_list AS id_map
        MATCH (e:`{BASE_ENTITY_LABEL}`)
        WHERE e.id = id_map.id
        WITH e, id_map.id AS id, id_map.seed_idx AS seed_idx  // Include seed_idx

        // For each id, find the top N most relevant relations
        CALL {{
            WITH e
            MATCH p=(e)-[r*1..{path_depth}]-(other)
            WHERE ALL(rel IN relationships(p) WHERE type(rel) <> 'MENTIONS')
            UNWIND relationships(p) AS rel
            WITH DISTINCT rel
            WHERE rel.embedding IS NOT NULL AND size(rel.embedding) = $dimension
            WITH rel, vector.similarity.cosine(rel.embedding, $embedding) AS rel_score
            ORDER BY rel_score DESC
            LIMIT toInteger(7)
            RETURN rel, rel_score
        }}

        WITH DISTINCT rel, rel_score, id AS seed_id, seed_idx  // Include seed_idx
        WITH startNode(rel) AS source,
            type(rel) AS type,
            rel{{.*}} AS rel_properties,
            endNode(rel) AS endNode,
            seed_id,
            seed_idx,  // Include seed_idx
            rel_score

        // Find the best matching text node for the source node
        CALL {{
            WITH source
            OPTIONAL MATCH (source)-[:MENTIONS]-(text_source)
            WHERE text_source.embedding IS NOT NULL AND size(text_source.embedding) = $dimension
            WITH text_source, vector.similarity.cosine(text_source.embedding, $embedding) AS source_score
            ORDER BY source_score DESC
            LIMIT 1
            RETURN text_source.id AS source_best_text_id, source_score AS source_best_score
        }}

        // Find the best matching text node for the end node
        CALL {{
            WITH endNode
            OPTIONAL MATCH (endNode)-[:MENTIONS]-(text_target)
            WHERE text_target.embedding IS NOT NULL AND size(text_target.embedding) = $dimension
            WITH text_target, vector.similarity.cosine(text_target.embedding, $embedding) AS target_score
            ORDER BY target_score DESC
            LIMIT 1
            RETURN text_target.id AS target_best_text_id, target_score AS target_best_score
        }}

        RETURN DISTINCT 
            source.id AS source_id,
            [l IN labels(source) WHERE NOT l IN ['{BASE_ENTITY_LABEL}', '{BASE_NODE_LABEL}'] | l][0] AS source_type,
            source{{.*, embedding: NULL, id: NULL, best_text_id: source_best_text_id, best_text_score: source_best_score }} AS source_properties,
            type,
            rel_properties,
            endNode.id AS target_id,
            [l IN labels(endNode) WHERE NOT l IN ['{BASE_ENTITY_LABEL}', '{BASE_NODE_LABEL}'] | l][0] AS target_type,
            endNode{{.*, embedding: NULL, id: NULL, best_text_id: target_best_text_id, best_text_score: target_best_score }} AS target_properties,
            rel_score,
            seed_id,
            seed_idx  // Include seed_idx

        ORDER BY seed_idx, rel_score DESC  // Order by seed_idx to maintain initial order
        """,
        param_map={
            "ids": ids,
            "limit": 30,
            "embedding": query.query_embedding,
            "dimension": len(query.query_embedding),
        },
    )      

        extracted_response_unsorted = []
        for record in response_unsorted:
            extracted_response_unsorted.append({
                "seed_id": record["seed_id"],
                "source_id": record["source_id"],
                "type": record["type"],
                "target_id": record["target_id"],
                "rel_score": record["rel_score"]
            })
        
        extracted_response_sorted = []
        for record in response_sorted:
            extracted_response_sorted.append({
                "seed_id": record["seed_id"],
                "source_id": record["source_id"],
                "type": record["type"],
                "target_id": record["target_id"],
                "rel_score": record["rel_score"]
            })
        print(extracted_response_unsorted)
        # if directory does not exist make it
        if not os.path.exists(f"OIA Nodes/full_unsorted_k4_d{path_depth}"):
            os.makedirs(f"OIA Nodes/full_unsorted_k4_d{path_depth}")
        with open(f"OIA Nodes/full_unsorted_k4_d{path_depth}/{i+1}.txt", 'w') as f:
            f.write(f"{query.query_str}\n")
            # do not print the keys of the dictionary just the values with a  head --[relation]--> tail
            # add the ids to know which ones are the seeds
            for id in ids:
                f.write(f"Seed: {id}\n")

            for item in extracted_response_unsorted:
                    f.write(f"[{item['seed_id']}]: {item['source_id']} --[{item['type']}]-> {item['target_id']} {item['rel_score']}\n")
        
        # the same for the sorted
        if not os.path.exists(f"OIA Nodes/full_sorted_k4_d{path_depth}"):
            os.makedirs(f"OIA Nodes/full_sorted_k4_d{path_depth}")
        with open(f"OIA Nodes/full_sorted_k4_d{path_depth}/{i+1}.txt", 'w') as f:
            f.write(f"{query.query_str}\n")
            # do not print the keys of the dictionary just the values with a  head --[relation]--> tail
            # add the ids to know which ones are the seeds
            for id in ids:
                f.write(f"Seed: {id}\n")

            for item in extracted_response_sorted:
                    f.write(f"[{item['seed_id']}]: {item['source_id']} --[{item['type']}]-> {item['target_id']} {item['rel_score']}\n")


What is the NZTA planning in terms of road network changes to accommodate the increased traffic from the new Interislander ferries in Marlborough?


Batches: 100%|██████████| 1/1 [00:00<00:00, 18.38it/s]

What is the NZTA planning in terms of road network changes to accommodate the increased traffic from the new Interislander ferries in Marlborough?


[{'seed_id': '2023-24 State Highway Investment Proposal', 'source_id': '2023-24 State Highway Investment Proposal', 'type': 'INCLUDES', 'target_id': 'Hornby Hub and Corridor Safety and Optimisation Single Stage Business Case', 'rel_score': 0.8258973360061646}, {'seed_id': '2023-24 State Highway Investment Proposal', 'source_id': 'Regional Land Transport Plan', 'type': 'ASSESSMENT_OF', 'target_id': 'Hornby Hub and Corridor Safety and Optimisation Single Stage Business Case', 'rel_score': 0.8242267966270447}, {'seed_id': '2023-24 State Highway Investment Proposal', 'source_id': 'Hornby Hub and Corridor Safety and Optimisation Single Stage Business Case', 'type': 'POTENTIAL_FOR', 'target_id': 'short-term and longer-term improvements', 'rel_score': 0.8085301518440247}, {'seed_id': '2023-24 State Highway Investment Proposal', 'source_id': '2023-24 State Highway Investment Proposal', 'type': 'INCLUDES', 'target_id': 'Hornby Access and Development Programme Business Case', 'rel_score': 0.8242

Batches: 100%|██████████| 1/1 [00:00<00:00, 59.40it/s]


...

How many crashes in Christchurch were alcohol-related between 2017 and 2019?


[{'seed_id': 'Road Traffic Crashes', 'source_id': 'Road Traffic Crashes', 'type': 'ANALYZED_BY', 'target_id': 'Crash Analysis', 'rel_score': 0.8155063390731812}, {'seed_id': 'Road Traffic Crashes', 'source_id': 'Crash Analysis', 'type': 'PERFORMED_BY', 'target_id': 'Waka Kotahi NZ Transport Agency', 'rel_score': 0.8289096355438232}, {'seed_id': 'Road Traffic Crashes', 'source_id': 'Waka Kotahi NZ Transport Agency', 'type': 'LOCATED_IN', 'target_id': 'Wellington', 'rel_score': 0.7657929062843323}, {'seed_id': 'Road Traffic Crashes', 'source_id': 'New Zealand Transport Agency', 'type': 'BASED', 'target_id': 'Wellington', 'rel_score': 0.790094792842865}, {'seed_id': 'Road Traffic Crashes', 'source_id': 'Private Bag 6995', 'type': 'LOCATED_IN', 'target_id': 'Wellington', 'rel_score': 0.7635517716407776}, {'seed_id': 'Road Traffic Crashes', 'source_id': 'Golden Mile', 'type': 'LOCATED', 'target_id': 'Wellington', 'rel_score': 0.7529628276824951}, {'seed_id': 'Road Traffic Crashes', 'source_

Batches: 100%|██████████| 1/1 [00:00<00:00, 63.36it/s]


When were the safety cameras in the Waterview Tunnel last calibrated for accuracy?
[{'seed_id': 'MINO -1042 Waterview Tunnel', 'source_id': 'MINO -1042 Waterview Tunnel', 'type': 'WARNING', 'target_id': '19 April 2024', 'rel_score': 0.8536930680274963}, {'seed_id': 'MINO -1042 Waterview Tunnel', 'source_id': '19 April 2024', 'type': 'RELEASED', 'target_id': 'Released in full', 'rel_score': 0.7138846516609192}, {'seed_id': 'MINO -1042 Waterview Tunnel', 'source_id': '29 April 2024', 'type': 'RELEASED', 'target_id': 'Released in full', 'rel_score': 0.7134437561035156}, {'seed_id': 'MINO -1042 Waterview Tunnel', 'source_id': 'MINO -1047 Driver instructors', 'type': 'TESTING', 'target_id': '29 April 2024', 'rel_score': 0.767119288444519}, {'seed_id': 'MINO -1042 Waterview Tunnel', 'source_id': '30 April 2024', 'type': 'RELEASED', 'target_id': 'Released in full', 'rel_score': 0.7116948366165161}, {'seed_id': 'MINO -1042 Waterview Tunnel', 'source_id': 'MINO -1051 Supporting information', 't

Batches: 100%|██████████| 1/1 [00:00<00:00, 57.40it/s]

In [52]:
nodes_with_score = property_graph_store.vector_query(query)

In [53]:
ids = []
for node in nodes_with_score[0]:
    ids.append(node.name)
ids

['reply', 'public interest', 'review', 'www.ombudsman.parliament.nz']

In [55]:
response_unsorted = property_graph_store.structured_query(
    f"""
    WITH $ids AS id_list
    UNWIND range(0, size(id_list) - 1) AS idx
    MATCH (e:`{BASE_ENTITY_LABEL}`)
    WHERE e.id = id_list[idx]
    MATCH p=(e)-[r*1..{path_depth}]-(other)
    WHERE ALL(rel IN relationships(p) WHERE type(rel) <> 'MENTIONS')
    UNWIND relationships(p) AS rel
    WITH DISTINCT rel, idx, id_list[idx] AS seed_id, vector.similarity.cosine(rel.embedding, $embedding) AS rel_score
    WITH startNode(rel) AS source,
         type(rel) AS type,
         rel{{.*}} AS rel_properties,
         endNode(rel) AS endNode,
         idx,
         rel_score,
         seed_id
    LIMIT toInteger($limit)
    RETURN source.id AS source_id, 
           [l IN labels(source) WHERE NOT l IN ['{BASE_ENTITY_LABEL}', '{BASE_NODE_LABEL}'] | l][0] AS source_type,
           source{{.*, embedding: Null, id: Null}} AS source_properties,
           type,
           rel_properties,
           endNode.id AS target_id, 
           [l IN labels(endNode) WHERE NOT l IN ['{BASE_ENTITY_LABEL}', '{BASE_NODE_LABEL}'] | l][0] AS target_type,
           endNode{{.*, embedding: Null, id: Null}} AS target_properties,
           idx,
           rel_score,
           seed_id
    ORDER BY idx
    LIMIT toInteger($limit)
    """,
    param_map={"ids": ids, "limit": 30, "embedding": query.query_embedding},
)

extracted_response_unsorted = []
for record in response_unsorted:
    extracted_response_unsorted.append({
        "seed_id": record["seed_id"],
        "source_id": record["source_id"],
        "type": record["type"],
        "target_id": record["target_id"],
        "rel_score": record["rel_score"]
    })
            
        
with open(f"Original Nodes OIA/1.txt", 'w') as f:
    f.write(f"{query.query_str}\n")
    # do not print the keys of the dictionary just the values with a  head --[relation]--> tail
    # add the ids to know which ones are the seeds
    for id in ids:
        f.write(f"Seed: {id}\n")

    for item in extracted_response_unsorted:
        f.write(f"[{item['seed_id']}]: {item['source_id']} --[{item['type']}]-> {item['target_id']} {item['rel_score']}\n")


In [38]:
response_unsorted

[{'source_id': 'Road safety initiatives',
  'source_type': 'entity',
  'source_properties': {'id': None,
   'creation_date': '2024-11-12',
   'last_modified_date': '2024-11-12',
   'file_size': 2707,
   'file_path': '/Users/felipenavarro/Documents/Auckland/nzta/nzta_experiment/NZTA-GraphRAG-Experiment/temp_passages/first_hops_Road_Safety_Initiatives.txt',
   'name': 'Road safety initiatives',
   'embedding': None,
   'file_name': 'first_hops_Road_Safety_Initiatives.txt',
   'triplet_source_id': '98f94edd-a81c-46e0-b44b-65c2c9152202',
   'file_type': 'text/plain'},
  'type': 'Focus on',
  'rel_properties': {'creation_date': '2024-11-12',
   'last_modified_date': '2024-11-12',
   'file_size': 2707,
   'file_path': '/Users/felipenavarro/Documents/Auckland/nzta/nzta_experiment/NZTA-GraphRAG-Experiment/temp_passages/first_hops_Road_Safety_Initiatives.txt',
   'file_name': 'first_hops_Road_Safety_Initiatives.txt',
   'triplet_source_id': '98f94edd-a81c-46e0-b44b-65c2c9152202',
   'file_type'

(200, 3)
